<a href="https://colab.research.google.com/github/worldterminator/mess/blob/main/trump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
file_path = '/content/drive/My Drive/trump.csv'
trump = pd.read_csv(file_path)

print(trump.head())

Mounted at /content/drive
   Durationinseconds                                      describephoto  \
0                 30                                                NaN   
1                142                                            rioting   
2                243   Terrorist acts against a duly elected government   
3                162        People are trying to overturn our democracy   
4                282  White supremacists under the guise of Qanon ar...   

                                          whynervous feedback     IP_country  \
0                                                NaN      NaN  United States   
1         Because it shows disregard for rule of law      NaN  United States   
2  Our president is encouraging a coup and has un...      NaN  United States   
3  I think they're trying to disregard the basis ...      NaN  United States   
4  It is worrying that the president has encourag...      NaN  United States   

   ID  
0  66  
1  67  
2  68  
3  69  
4 

In [12]:
trump.dropna(subset=['whynervous'], inplace=True)  # dropping rows without a response
# Basic text cleaning (lowercasing, removing punctuation)
trump['whynervous_clean'] = trump['whynervous'].str.lower().str.replace('[^\w\s]', '', regex=True)

min_words = 3
trump['flag_invalid'] = trump['whynervous_clean'].apply(lambda x: len(x.split()) < min_words)
print(trump)

      Durationinseconds                                      describephoto  \
1                   142                                            rioting   
2                   243   Terrorist acts against a duly elected government   
3                   162        People are trying to overturn our democracy   
4                   282  White supremacists under the guise of Qanon ar...   
5                   243  The people in the photo are attempting to brea...   
...                 ...                                                ...   
1266               3196  They are being sprayed by police while seeming...   
1267                577                            Protesting Trumps loss.   
1268                331  taking their country back and taking action to...   
1269               1366        Reacting negatively to the election outcome   
1277                323                                  an act of terror.   

                                             whynervous      fe

In [13]:
summary_stats = trump['flag_invalid'].describe()
invalid_valid_counts = trump['flag_invalid'].value_counts()
print(summary_stats)
print(invalid_valid_counts)

count      1180
unique        2
top       False
freq       1157
Name: flag_invalid, dtype: object
False    1157
True       23
Name: flag_invalid, dtype: int64


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download necessary NLTK data (do this once)
nltk.download('punkt')

# Tokenizing responses
data['whynervous_tokens'] = data['whynervous_clean'].apply(word_tokenize)

# Example of further analysis: Counting tokens in each response
data['token_count'] = data['whynervous_tokens'].apply(len)
# Filtering rows where the token count is 3 or more
data_filtered = data[data['token_count'] >= 3]


removing stop words might be less necessary for flagging bad responses but could help preparing the filtered data for later analysis...codes that include re

In [ ]:
from nltk.corpus import stopwords
import string

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Set of English stop words
stop_words = set(stopwords.words('english'))

# Function to tokenize and clean text (remove punctuation and stop words)
def clean_tokenize(text):
    # Removing punctuation
    no_punctuation = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizing
    tokens = word_tokenize(no_punctuation)
    # Removing stop words and lowercasing
    return [word.lower() for word in tokens if word.lower() not in stop_words]

# Applying the cleaning and tokenization function
data['whynervous_tokens_clean'] = data['whynervous_clean'].apply(clean_tokenize)

# Counting tokens again (after removing stop words and punctuation)
data['token_count_clean'] = data['whynervous_tokens_clean'].apply(len)

# Filtering rows based on the new token count
data_filtered = data[data['token_count_clean'] >= 3]
